In [1]:
## load in relevant packages
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(1)

In [902]:
##===================================================================================
## read in data

## winter
## collapsed over time
#dta = pd.read_csv("../data/kcl_london_model_data_winter_collapsed.csv", sep=',') 
## aggregated over time
dta = pd.read_csv("../data/kcl_london_model_data_winter_agg_time.csv", sep=',')

## not winter
## collapsed over time
#dta = pd.read_csv("../data/kcl_london_model_data_nowinter_collapsed.csv", sep=',')
## aggregated over time
#dta = pd.read_csv("../data/kcl_london_model_data_nowinter_agg_time.csv", sep=',')

## monthly data (2000-2019)
#dta = pd.read_csv("../data/kcl_london_model_data_monthly.csv", sep=',')
## subset to only one month (1-12)
#dta = dta[dta['month']==5]

## set variables to use in model
params = ['latitude', 'longitude', 'year']
#params = ['latitude', 'longitude']
ncols= len(params)

##===================================================================================

In [903]:
## preview data
print(dta.shape)
print(dta.head())

## standardize X-values (or should I just subtract 52 from latitude and leave longitude? 
#if 'year' in dta.columns:
#    dta[['year']] = dta[['year']] - np.min(dta[['year']])
#if 'latitude' in dta.columns:
#    dta[['latitude']] = dta[['latitude']] - 52

## divide into features and variable
X = dta[params].values
y = dta.loc[:,'nox'].values
y = y.reshape(-1,1)

print(X.shape)
print(y.shape)

## print previews
print(y[0:10])
print(X[1:10,:])

(1981, 7)
                            site code   latitude  longitude   site_type  year  \
0               Heathrow Airport  LH2  51.479234  -0.440531  Industrial  2000   
1       Barnet - Tally Ho Corner  BN1  51.614675  -0.176607    Kerbside  2000   
2         Camden - Swiss Cottage  CD1  51.544219  -0.175284    Kerbside  2000   
3  Westminster - Marylebone Road  MY1  51.522540  -0.154590    Kerbside  2000   
4              Croydon - Norbury  CR5  51.411349  -0.123110    Kerbside  2000   

          nox  
0  148.090848  
1  199.026427  
2  217.054604  
3  457.892734  
4  235.985324  
(1981, 3)
(1981, 1)
[[148.09084761]
 [199.0264266 ]
 [217.05460423]
 [457.89273426]
 [235.98532394]
 [433.92134367]
 [239.16108482]
 [167.38472428]
 [184.71412974]
 [161.47389498]]
[[ 5.16146750e+01 -1.76607000e-01  2.00000000e+03]
 [ 5.15442190e+01 -1.75284000e-01  2.00000000e+03]
 [ 5.15225400e+01 -1.54590000e-01  2.00000000e+03]
 [ 5.14113490e+01 -1.23110000e-01  2.00000000e+03]
 [ 5.15583462e+01  6.9

In [905]:
from sklearn.preprocessing import StandardScaler  
feature_scaler = StandardScaler() 

#if X.shape[1] >= 3:
    ## rescale lat/long and year data
#    X[:,0:1] = feature_scaler.fit_transform(X[:,0:1])
    ## rescale year data from 0 to 1
#    X[:,2] = (X[:,2] - min(X[:,2])) / (max(X[:,2]) - min(X[:,2]))
#else:

## rescale lat/long and year data
X = feature_scaler.fit_transform(X)
#X[:,0] = X[:,0] - np.mean(X[:,0])
#X[:,1] = X[:,1] - np.mean(X[:,1])
#X[:,2] = X[:,2] - np.mean(X[:,2])

In [906]:
## create validation dataset (no test set since using MLL)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0) 

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(1485, 3)
(1485, 1)
(496, 3)
(496, 1)


In [907]:
## Normalize Y (after splitting into training and validation)

## standardize y-values
y_train = feature_scaler.fit_transform(y_train)
y_val = feature_scaler.fit_transform(y_val)

print(y_train.shape)
print(y_val.shape)

(1485, 1)
(496, 1)


In [908]:
## build GP model
import gpflow

## set kernel
## 0: RBF(lat,long)
## 1: RBF(lat,long) * RBF(year)
## 2: RBF(lat,long) * Linear(year)
## 3: RBF(lat,long) * Polynomial(year)
which_kernel = 3
which_iter = 1

if (which_kernel == 1) & ('year' not in params):
        print("ERROR: did not specify YEAR as parameter!!")

In [909]:
## do I need ARD?
## "In our case, we will use the Squared Exponential covariance function for f,
## and an Automatic Relevence Determination version of the SE covariance function for g.
## The ARD will allow us to find which predictor variables affect predictions from the model,
## which should relate to their importance."

if which_kernel == 0:
    kernel = gpflow.kernels.RBF(2, active_dims=[0,1], lengthscales=1.0)
if which_kernel == 1:
    kernel = gpflow.kernels.RBF(2, active_dims=[0,1], lengthscales=1.0) *\
        gpflow.kernels.RBF(1 , active_dims=[2], lengthscales=0.1)
if which_kernel == 2:
    kernel = gpflow.kernels.RBF(2, active_dims=[0,1], lengthscales=1.0) *\
        gpflow.kernels.Linear(1 , active_dims=[2])
if which_kernel == 3:
    kernel = gpflow.kernels.RBF(2, active_dims=[0,1], lengthscales=1.0) *\
                gpflow.kernels.Polynomial(1, degree=2., active_dims=[2])
        
    
## ## option #3) all data
## periodic kernel in time (every month) * linear/polynomial (linear trend) * squared exponential in space

## Modeling categorical data
## There is a simple way to do GP regression over categorical variables.
## Simply represent your categorical variable as a by a one-of-k encoding.
## This means that if your number ranges from 1 to 5, represent that as 5 different data dimensions,
## only one of which is on at a time. 
## Then, simply put a product of SE kernels on those dimensions.
## This is the same as putting one SE ARD kernel on all of them.
## The lengthscale hyperparameter will now encode whether, when that coding is active,
## the rest of the function changes.
## If you notice that the estimated lengthscales for your categorical variables is short,
## your model is saying that it's not sharing any information between data of different categories. 


    
## build model
m = gpflow.models.GPR(X_train, y_train, kern=kernel)
m.likelihood.variance = 0.01

## view 
m.as_pandas_table()

,class,prior,transform,trainable,shape,fixed_shape,value
GPR/kern/kernels/0/lengthscales,Parameter,None,+ve,True,(),True,1.0
GPR/kern/kernels/0/variance,Parameter,None,+ve,True,(),True,1.0
GPR/kern/kernels/1/offset,Parameter,None,+ve,True,(),True,1.0
GPR/kern/kernels/1/variance,Parameter,None,+ve,True,(),True,1.0
GPR/likelihood/variance,Parameter,None,+ve,True,(),True,0.01


In [910]:
## Run Model
## Marginal Liklihood Maximization
## picks the most simple model that picks the data the best
gpflow.train.ScipyOptimizer().minimize(m)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 735.950639
  Number of iterations: 20
  Number of functions evaluations: 25


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 735.950639
  Number of iterations: 20
  Number of functions evaluations: 25


In [911]:
print(m.as_pandas_table())

                                     class prior transform  trainable shape  \
GPR/kern/kernels/0/lengthscales  Parameter  None       +ve       True    ()   
GPR/kern/kernels/0/variance      Parameter  None       +ve       True    ()   
GPR/kern/kernels/1/offset        Parameter  None       +ve       True    ()   
GPR/kern/kernels/1/variance      Parameter  None       +ve       True    ()   
GPR/likelihood/variance          Parameter  None       +ve       True    ()   

                                 fixed_shape                  value  
GPR/kern/kernels/0/lengthscales         True  0.0009381454730469734  
GPR/kern/kernels/0/variance             True     0.5323843265000349  
GPR/kern/kernels/1/offset               True     1.4535385097676954  
GPR/kern/kernels/1/variance             True    0.10094021252427628  
GPR/likelihood/variance                 True     0.0651536723646352  


In [912]:
## score model
mean, var = m.predict_y(X_val)

In [913]:
interval_95 = 2*np.sqrt(var[:,0])
interval_95 = np.expand_dims(interval_95, axis=1)
top_interval_95 = mean + interval_95
bottom_interval_95 = mean - interval_95
## evaluate whether point falls in interval
truth = []
for i in range(y_val.shape[0]):
    if y_val[i,0] >= bottom_interval_95[i,0] and y_val[i,0] <= top_interval_95[i,0]:
        truth.append(1)
    else:
        truth.append(0)
print(np.sum(truth))
score = np.sum(truth) / y_val.shape[0]
print(score)

478
0.9637096774193549


In [914]:
## also calculate MSE
## number of points
n = mean.shape[0]
diff = (y_val[:,0] - mean[:,0])**2
mse = np.sum(diff) / n
print(mse)

0.06987115996250357


In [ ]:
## Questions for Stefanie:

## 1) How to score models? Currently evaluating whether the 95% confidence interval captures the actual value for
## the validation set. How to balance the fact that model intervals are too wide?
## 2) How to visualize results in 2 and 3-d?

In [889]:
score_wc_m0 = []

mse_wc_m0 = []

In [890]:
## Winter
score_wc_m0.append(0.9482758620689655)
mse_wc_m0.append(0.889188407571395)

## Not Winter
score_wc_m0.append(0.9482758620689655)
mse_wc_m0.append(0.944077087789119)

In [892]:
print(np.mean(score_wc_m0))
print(np.mean(mse_wc_m0))

0.9482758620689655
0.9166327476802569


In [597]:
#score_w_m0 = []
score_w_m1 = []
score_w_m2 = []
score_w_m3 = []

#mse_w_m0 = []
mse_w_m1 = []
mse_w_m2 = []
mse_w_m3 = []

In [598]:
## Winter
score_w_m1.append(0.9596774193548387)
score_w_m2.append(0.9556451612903226)
score_w_m3.append(0.9637096774193549)

mse_w_m1.append(0.9070767715745317)
mse_w_m2.append(0.15715599665719784)
mse_w_m3.append(0.06987115996250357)

## Non-Winter
score_w_m1.append(0.9666666666666667)
score_w_m2.append(0.96)
score_w_m3.append(0.9555555555555556)

mse_w_m1.append(0.8919486748914799)
mse_w_m2.append(0.1379321844772651)
mse_w_m3.append(0.10993582155549385)

In [603]:
print(np.mean(score_w_m1))
print(np.mean(score_w_m2))
print(np.mean(score_w_m3))

print(np.mean(mse_w_m1))
print(np.mean(mse_w_m2))
print(np.mean(mse_w_m3))

0.9631720430107527
0.9578225806451612
0.9596326164874552
0.8995127232330058
0.14754409056723147
0.0899034907589987


In [925]:
#score_all_m0 = []
score_all_m1 = []
score_all_m2 = []
score_all_m3 = []

#mse_all_m0 = []
mse_all_m1 = []
mse_all_m2 = []
mse_all_m3 = []

In [926]:
## January
score_all_m1.append(0.9522727272727273)
mse_all_m1.append(0.8456863378328031)

score_all_m2.append(0.9295454545454546)
mse_all_m2.append(0.293738121763502)

score_all_m3.append(0.9318181818181818)
mse_all_m3.append(0.28575088342736316)

## Febrary
score_all_m1.append(0.9501133786848073)
mse_all_m1.append(0.9498865601879187)

score_all_m2.append(0.9433106575963719)
mse_all_m2.append(0.45728805929725613)

score_all_m3.append(0.9002267573696145)
mse_all_m3.append(1.6687370138120152)

## March
score_all_m1.append(0.9641255605381166)
mse_all_m1.append(0.8622644623424907)

score_all_m2.append(0.9551569506726457)
mse_all_m2.append(0.9980547846555808)

score_all_m3.append(0.9349775784753364)
mse_all_m3.append(0.20234739028743054)

## April
score_all_m1.append(0.9577464788732394)
mse_all_m1.append(0.938434528957358)

score_all_m2.append(0.9530516431924883)
mse_all_m2.append(0.14154699613486757)

score_all_m3.append(0.9436619718309859)
mse_all_m3.append(0.12071800470906409)

## May
score_all_m1.append(0.9502369668246445)
mse_all_m1.append(0.8891895921499919)

score_all_m2.append(0.9597156398104265)
mse_all_m2.append(0.1458077967763497)

score_all_m3.append(0.9597156398104265)
mse_all_m3.append(0.11270673779635344)

## June
score_all_m1.append(0.9522673031026253)
mse_all_m1.append(0.9099270364887586)

score_all_m2.append(0.9379474940334129)
mse_all_m2.append(0.3122744859095631)

score_all_m3.append(0.9212410501193318)
mse_all_m3.append(0.2643669979671157)

## July
score_all_m1.append(0.9527186761229315)
mse_all_m1.append(0.9036238395028933)

score_all_m2.append(0.9361702127659575)
mse_all_m2.append(0.1588594575454301)

score_all_m3.append(0.9432624113475178)
mse_all_m3.append(0.10461483930076629)

## August
score_all_m1.append(0.9474940334128878)
mse_all_m1.append(0.9413148346599836)

score_all_m2.append(0.9474940334128878)
mse_all_m2.append(0.19108466579058989)

score_all_m3.append(0.9451073985680191)
mse_all_m3.append(0.1414502763958607)

## September
score_all_m1.append(0.9407582938388626)
mse_all_m1.append(1.0006019083124036)

#score_all_m2.append(np.nan)
#mse_all_m2.append(np.nan)

score_all_m3.append(0.966824644549763)
mse_all_m3.append(0.11038204107636336)

## October
score_all_m1.append(0.9529411764705882)
mse_all_m1.append(0.9437203826510108)

score_all_m2.append(0.9458823529411765)
mse_all_m2.append(0.2656939430133758)

score_all_m3.append(0.9576470588235294)
mse_all_m3.append(0.21791853695775756)

## November
score_all_m1.append(0.9598108747044918)
mse_all_m1.append(0.881269262930133)

score_all_m2.append(0.9432624113475178)
mse_all_m2.append(0.3799254185984513)

score_all_m3.append(0.9432624113475178)
mse_all_m3.append(0.3481292191320313)

## December
score_all_m1.append(0.950354609929078)
mse_all_m1.append(0.865659369444384)

#score_all_m2.append(np.nan)
#mse_all_m2.append(np.nan)

score_all_m3.append(0.9432624113475178)
mse_all_m3.append(0.25517511767401957)

In [927]:
print(np.mean(score_all_m1))
print(np.mean(score_all_m2))
print(np.mean(score_all_m3))

print(np.mean(mse_all_m1))
print(np.mean(mse_all_m2))
print(np.mean(mse_all_m3))

0.9525700066479167
0.945153685031834
0.9409172929506453
0.9109648429550107
0.3344273729484966
0.3193580882113451


In [929]:
## plot accuracies for each month and model

print(mse_all_m3)

[0.28575088342736316, 1.6687370138120152, 0.20234739028743054, 0.12071800470906409, 0.11270673779635344, 0.2643669979671157, 0.10461483930076629, 0.1414502763958607, 0.11038204107636336, 0.21791853695775756, 0.3481292191320313, 0.25517511767401957]


In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import cross_val_score

In [ ]:
import gpflow
import numpy as np
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
plt = matplotlib.pyplot

X = np.linspace(-3,3,20)
Y = np.random.exponential(np.sin(X)**2)

with gpflow.defer_build():
    k = gpflow.kernels.Matern32(1, ARD=False) + gpflow.kernels.Bias(1)
    l = gpflow.likelihoods.Exponential()
    m = gpflow.models.GPMC(X[:,None], Y[:,None], k, l)

m.kern.kernels[0].lengthscales.prior = gpflow.priors.Gamma(1., 1.)
#m.kern.kernels[1].lengthscales.prior = gpflow.priors.Gamma(1., 1.)
#m.kernels.Bias.variance.prior = gpflow.priors.Gamma(1.,1.)
#m.kernels.Matern32.lengthscales.prior = gpflow.priors.Gamma(1., 1.)
#m.kernels.Matern32.variance.prior = gpflow.priors.Gamma(1.,1.)
#m.kernels.Bias.variance.prior = gpflow.priors.Gamma(1.,1.)

m.compile()
o = gpflow.train.AdamOptimizer(0.01)
o.minimize(m, maxiter=15) # start near MAP

s = gpflow.train.HMC()
samples = s.sample(m, 100, epsilon=0.12, lmax=20, lmin=5, thin=5, logprobs=False)#, verbose=True)
samples.head()


In [ ]:
print(np.mean(samples.iloc[:,1]))
print(np.var(samples.iloc[:,1]))
print(np.mean(samples.iloc[:,2]))
print(np.var(samples.iloc[:,2]))

In [ ]:
# make a more informative plot
#plt.figure(figsize=(16, 4))
#for lab, s in samples.iteritems():
#    plt.plot(s, label=lab)
#_ = plt.legend(loc=0)




#for col in samples.columns.sort_values()[1:]:
#    samples[col].hist(label=col.split('.')[-1], alpha=0.4, bins=15)

In [ ]:
xtest = np.linspace(-4,4,100)[:,None]
f_samples = []
for i, s in samples.iterrows():
    m.assign(s)
    f_samples.append(m.predict_f_samples(xtest, 5, initialize=False))
f_samples = np.vstack(f_samples)

In [ ]:
rate_samples = np.exp(f_samples[:, :, 0])

line, = plt.plot(xtest, np.mean(rate_samples, 0), lw=2)
plt.fill_between(xtest[:,0],
                 np.percentile(rate_samples, 5, axis=0),
                 np.percentile(rate_samples, 95, axis=0),
                 color=line.get_color(), alpha = 0.2)

plt.plot(X, Y, 'kx', mew=2)
plt.ylim(-0.1, np.max(np.percentile(rate_samples, 95, axis=0)))

In [ ]:
opt = gpflow.train.ScipyOptimizer()
opt.minimize(m)

In [ ]:
N = 12
X = np.random.rand(N,1)
Y = np.sin(12*X) + 0.66*np.cos(25*X) + np.random.randn(N,1)*0.1 + 3
print(Y.shape)
print(X.shape)
plt.plot(X, Y, 'kx', mew=2)

k = gpflow.kernels.Matern52(1, lengthscales=0.3)
m = gpflow.models.GPR(X, Y, kern=k)
m.likelihood.variance = 0.01
m.compile()

In [ ]:
def plot(m):
    xx = np.linspace(-0.1, 1.1, 100)[:,None]
    mean, var = m.predict_y(xx)
    plt.figure(figsize=(12, 6))
    plt.plot(X, Y, 'kx', mew=2)
    plt.plot(xx, mean, 'b', lw=2)
    plt.fill_between(xx[:,0], mean[:,0] - 2*np.sqrt(var[:,0]), mean[:,0] + 2*np.sqrt(var[:,0]), color='blue', alpha=0.2)
    plt.xlim(-0.1, 1.1)
plot(m)

In [ ]:
from scipy.spatial import distance as d

print(X_train[0:10,:])

space_length_scale = 1.0
time_length_scale = 1.0

#X_space = X_train[:,0:2]

X_space = np.atleast_2d(X_train[0,0:2])
Y_space = np.atleast_2d(X_train[1,0:2])
X_time = np.atleast_2d(X_train[0,2])
Y_time = np.atleast_2d(X_train[1,2])
print(X_space[0:5,:])
print(X_space.shape)
print(X_time[0:5,:])
print(X_time.shape)


dists_space = d.cdist(X_space / space_length_scale, Y_space / space_length_scale, metric='sqeuclidean')
dists_time = d.cdist(X_time / time_length_scale, Y_time / time_length_scale, metric='sqeuclidean')
K = np.exp(-.5 * dists_space) * np.exp(-.5 * dists_time)

print(dists_space)
print(dists_time)
print(K)


dists_space = d.pdist(X_space / space_length_scale, metric='sqeuclidean')
dists_time = d.pdist(X_time / time_length_scale, metric='sqeuclidean')
K = np.exp(-.5 * dists_space) * np.exp(-.5 * dists_time)
# convert from upper-triangular matrix to square matrix
K = d.squareform(K)
np.fill_diagonal(K, 1)

print(dists_space)
print(dists_time)
print(K)

In [ ]:
## ?? Should I specify a constant kernel?
kernel = RBF(length_scale=1)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1, normalize_y=True, n_restarts_optimizer=5)
all_accuracies = cross_val_score(estimator=gp, X=X_train, y=y_train, cv=5, scoring='r2').mean()
print(all_accuracies)
## winter data
## sigma = 1000, alpha=1
## 0.8037412498178955
## sigma = 1, alpha=1
## 0.8055139240920436
## sigma=1, alpha = 100
## 0.14285243700525588
## sigma=10, alpha=100
## 0.14285243655664298

In [ ]:
## search for best hyperparameters: sigma
all_accuracies_mean = []
all_accuracies_std = []
sigmas = np.arange(start=0.1, stop=4.5, step=0.5)
for sigma in sigmas:
    kernel = RBF(length_scale=sigma)
    gp = GaussianProcessRegressor(kernel=kernel, alpha=1, normalize_y=True, n_restarts_optimizer=5)
    all_accuracies = cross_val_score(estimator=gp, X=X_train, y=y_train, cv=5, scoring='r2')
    all_accuracies_mean.append(all_accuracies.mean())
    all_accuracies_std.append(all_accuracies.std())
print(sigmas)
print(all_accuracies_mean)
print(all_accuracies_std)

In [ ]:
## function to select the best parameter
def best_parameter (mean_acc, param_list):
    ## take sd of all accuracies, if small, then take the median sigma
    acc_sd = np.std(mean_acc)
    if acc_sd < 0.005:
        best_param = np.median(param_list)
    else:
        ## pick the sigma with the highest R^2
        best_param = param_list[np.argmax(mean_acc)]
    
        ## if large sd and tie, default to larger sigma (more variance)
        ## or could select the default value (1)
        #best_param = 1

    return(best_param)

In [ ]:
## find best parameter for sigma
best_sigma = best_parameter(all_accuracies_mean, sigmas)
print(best_sigma)

In [ ]:
## search for best hyperparameters: alpha
all_accuracies_al_mean = []
all_accuracies_al_std = []
alphas = np.arange(start=0.1, stop=1.1, step=0.1)
for alpha in alphas:
    kernel = RBF(length_scale=best_sigma)
    gp = GaussianProcessRegressor(kernel=kernel, alpha=alpha, normalize_y=True, n_restarts_optimizer=5)
    all_accuracies_al = cross_val_score(estimator=gp, X=X_train, y=y_train, cv=5, scoring='r2')
    all_accuracies_al_mean.append(all_accuracies_al.mean())
    all_accuracies_al_std.append(all_accuracies_al.std())
print(alphas)
print(all_accuracies_al_mean)
print(all_accuracies_al_std)

In [ ]:
## select the best alpha
best_alpha = best_parameter(all_accuracies_al_mean, alphas)
print(best_alpha)